In [2]:
# Cleaning UserAchievements.csv file
import pandas as pd
import os
import numpy as np

# Load the UserAchievements.csv file
df = pd.read_csv('/kaggle/input/meta-kaggle/UserAchievements.csv')
print("Reading completed..\n")

# Filter out rows where Points, TotalGold, and TotalSilver are all zero
df_filtered = df[~((df['Points'] == 0) & (df['TotalGold'] == 0) & (df['TotalSilver'] == 0))]
print("Filtering completed..\n")

# Save the filtered DataFrame to a new CSV file in the Kaggle working directory
df_filtered.to_csv('/kaggle/working/UserAchievements_Cleaned.csv', index=False)
print("Saving to --> /kaggle/working/UserAchievements_Cleaned.csv")

Reading completed..

Filtering completed..

Saving to --> /kaggle/working/UserAchievements_Cleaned.csv


In [ ]:
#Getting Metadata of the files

import pandas as pd
import os
import numpy as np

# Define the directory where your dataset is located
dataset_directory = '/kaggle/input/meta-kaggle/'  # Update this path to your specific dataset directory

# Output directory for Excel files
output_directory = '/kaggle/working/'  # This is a common output directory in Kaggle kernels

# List of specific filenames you want to process
filenames_to_process = [
    "UserAchievements_Cleaned.csv",
    "Submissions.csv",
    "Users.csv",
    "ForumMessages.csv",
    "Teams.csv",
    "UserFollowers.csv",
    "ForumMessageVotes.csv",
    "ForumTopics.csv",
    "KernelTags.csv",
    "Datasets.csv",
    "DatasetVersions.csv",
    "DatasetTags.csv",
    "Forums.csv",
    "Competitions.csv",
    "DatasetTaskSubmissions.csv",
    "DatasetTasks.csv",
    "UserOrganizations.csv",
    "Tags.csv",
    "Organizations.csv",
    "CompetitionTags.csv",
    "KernelLanguages.csv"
]
#tables = []
metadata = []

for filename in filenames_to_process:
    print(f"Processing {filename}...")
    # Special case for the cleaned UserAchievements.csv since it's in a different directory.
    if filename == "UserAchievements_Cleaned.csv":
        df = pd.read_csv(os.path.join(output_directory, filename))
    else:
        df = pd.read_csv(os.path.join(dataset_directory, filename))
    table_name = filename.split('.')[0]

    # Iterate through each column to gather metadata
    for column in df.columns:
        col_data = df[column]
        meta = {
            'table_name': table_name,
            'column_name': column,
            'data_type': col_data.dtype,
            'non_null_count': col_data.notnull().sum(),
            'unique_count': col_data.nunique()
        }

        # Additional statistics for numerical columns
        if pd.api.types.is_numeric_dtype(col_data):
            meta['min'] = col_data.min()
            meta['max'] = col_data.max()
            meta['mean'] = col_data.mean()
            meta['median'] = col_data.median()
            meta['std_dev'] = col_data.std()

        # Additional information for categorical columns
        if pd.api.types.is_categorical_dtype(col_data) or pd.api.types.is_object_dtype(col_data):
            top_categories = col_data.value_counts().nlargest(5).index.tolist()
            meta['top_categories'] = top_categories

        metadata.append(meta)

    #tables.append(df)
    print(f"Completed processing {filename}.\n")

# Concatenate all tables horizontally (you can change axis to 0 for vertical concatenation)
#result = pd.concat(tables, axis=1)

# Convert metadata list to a DataFrame for easier viewing and analysis
metadata_df = pd.DataFrame(metadata)

# Save the concatenated data and metadata to Excel files
#result_file_path = os.path.join(output_directory, 'concatenated_data.xlsx')
metadata_file_path = os.path.join(output_directory, 'metadata.xlsx')

#print("Saving concatenated data to Excel...")
#result.to_excel(result_file_path, index=False)

print("Saving metadata to Excel...")
metadata_df.to_excel(metadata_file_path, index=False)

print(f"Metadata file created successfully at : {metadata_file_path}\n")


In [2]:
# Cleaning Submissions.csv file
import pandas as pd

file_path = '/kaggle/input/meta-kaggle/Submissions.csv'
output_path = '/kaggle/working/Submissions_Cleaned.csv'

# Load the CSV file with only the specified columns
columns = ['Id', 'SubmittedUserId', 'TeamId', 'PublicScoreLeaderboardDisplay', 'SubmissionDate', 'IsAfterDeadline']
df = pd.read_csv(file_path, usecols=columns)

# Convert 'SubmissionDate' to datetime format to ensure correct sorting
df['SubmissionDate'] = pd.to_datetime(df['SubmissionDate'])

# Sort by 'SubmittedUserId' and 'SubmissionDate' to get the latest submission by each user
df_sorted = df.sort_values(by=['SubmittedUserId', 'SubmissionDate'], ascending=[True, False])

# Drop duplicate 'SubmittedUserId', keeping the first (latest submission based on 'SubmissionDate')
df_cleaned = df_sorted.drop_duplicates(subset='SubmittedUserId', keep='first')

# Save the cleaned DataFrame to a new CSV file
df_cleaned.to_csv(output_path, index=False)

print(f'Cleaned file saved to: {output_path}')

Cleaned file saved to: /kaggle/working/Submissions_Cleaned.csv


In [7]:
# Cleaning UserFollowers.csv file
import pandas as pd

# Load the UserFollowers.csv file
file_path = '/kaggle/input/meta-kaggle/UserFollowers.csv'
output_path = '/kaggle/working/UserFollowers_Cleaned.csv'

# Assuming the fields 'UserId', 'FollowingUserId', and 'CreationDate' exist in your dataset
df = pd.read_csv(file_path)

# Convert 'CreationDate' to datetime to ensure correct sorting
df['CreationDate'] = pd.to_datetime(df['CreationDate'])

# First, sort by 'UserId' and 'CreationDate' to prepare for dropping duplicates
df_sorted = df.sort_values(by=['UserId', 'CreationDate'], ascending=[True, False])

# Drop duplicates based on 'UserId' and 'FollowingUserId', keeping the latest entry
df_deduplicated = df_sorted.drop_duplicates(subset=['UserId', 'FollowingUserId'], keep='first')

# Now, create a new DataFrame to count followers per 'UserId'
df_followers_count = df_deduplicated.groupby('UserId').size().reset_index(name='Followers_Count')

# Save the cleaned and aggregated DataFrame to a new CSV file
df_followers_count.to_csv(output_path, index=False)

print(f'Cleaned file with followers count saved to: {output_path}')


Cleaned file with followers count saved to: /kaggle/working/UserFollowers_Cleaned.csv


In [3]:
# Cleaning Teams.csv file
import pandas as pd

# Load Teams.csv
teams_file_path = '/kaggle/input/meta-kaggle/Teams.csv'
teams = pd.read_csv(teams_file_path, usecols=[
    'Id', 'CompetitionId', 'TeamLeaderId', 'TeamName', 'LastSubmissionDate', 
    'PublicLeaderboardSubmissionId', 'Medal', 'PublicLeaderboardRank'
])

# Load Submissions_Cleaned.csv
submissions_cleaned_file_path = '/kaggle/working/Submissions_Cleaned.csv'
submissions_cleaned = pd.read_csv(submissions_cleaned_file_path)

# Filter out rows from Teams where the specified columns are all null
teams_cleaned = teams.dropna(
    subset=['LastSubmissionDate', 'PublicLeaderboardSubmissionId', 'Medal', 'PublicLeaderboardRank'],
    how='all'
)

# Ensure that Id from Teams is not present as TeamId in Submissions_Cleaned
teams_cleaned = teams_cleaned[~teams_cleaned['Id'].isin(submissions_cleaned['TeamId'])]

# Save the cleaned Teams data
output_path = '/kaggle/working/Teams_Cleaned.csv'
teams_cleaned.to_csv(output_path, index=False)

print(f"Cleaned Teams data saved to {output_path}")


Cleaned Teams data saved to /kaggle/working/Teams_Cleaned.csv


In [4]:
# Cleaning Teams.csv file
import pandas as pd
import re

# Paths for the files
teams_file_path = '/kaggle/input/meta-kaggle/Teams.csv'
submissions_cleaned_file_path = '/kaggle/working/Submissions_Cleaned.csv'
output_path = '/kaggle/working/Teams_Cleaned1.csv'

# Load the Teams.csv file
teams = pd.read_csv(teams_file_path, usecols=[
    'Id', 'CompetitionId', 'TeamLeaderId', 'TeamName', 'LastSubmissionDate', 
    'PublicLeaderboardSubmissionId', 'Medal', 'PublicLeaderboardRank'
])

# Load the Submissions_Cleaned.csv file
submissions_cleaned = pd.read_csv(submissions_cleaned_file_path)

# Define the regex pattern for allowed characters in TeamName
# Add or remove characters from the pattern as needed
allowed_chars_pattern = re.compile(r'^[a-zA-Z0-9\s\-#@$_,\']+$')

# Apply filters to the teams DataFrame
teams_cleaned = teams[
    # Filter out '[Deleted]' in TeamName
    ~teams['TeamName'].str.contains('\[Deleted\]', na=False) &
    # Keep only rows with allowed characters in TeamName
    teams['TeamName'].apply(lambda x: bool(allowed_chars_pattern.match(x)) if pd.notna(x) else True) &
    # Filter out null columns
    teams[['LastSubmissionDate', 'PublicLeaderboardSubmissionId', 'Medal', 'PublicLeaderboardRank']].notnull().any(axis=1)
]

# Ensure that Id from Teams is not present as TeamId in Submissions_Cleaned
teams_cleaned = teams_cleaned[~teams_cleaned['Id'].isin(submissions_cleaned['TeamId'])]

# Save the cleaned Teams data to a new CSV file
teams_cleaned.to_csv(output_path, index=False)

print(f"Cleaned Teams data saved to {output_path}")


Cleaned Teams data saved to /kaggle/working/Teams_Cleaned1.csv


In [ ]:
# Calculating sentiment polarity for messages 
import pandas as pd
from nltk.sentiment import SentimentIntensityAnalyzer

# Load your dataset
df = pd.read_csv('/kaggle/input/meta-kaggle/ForumMessages.csv')
output_path = '/kaggle/working/ForumMessages_Cleaned.csv'

# Drop duplicates across the entire dataframe
df = df.drop_duplicates()

# Convert 'MessageDate' to date format and remove the time part
df['MessageDate'] = pd.to_datetime(df['MessageDate']).dt.date

# Initialize the VADER sentiment intensity analyzer
sid = SentimentIntensityAnalyzer()

# Define a function to get the compound polarity score
def get_compound_polarity(text):
    try:
        scores = sid.polarity_scores(str(text))  # Ensure text is treated as a string
        return scores['compound']
    except Exception as e:
        print(f"Error processing text: {e}")
        return None

# Apply the function to your message column to create a new 'polarity_score' column
df['polarity_score'] = df['Message'].apply(get_compound_polarity)

# Drop the 'Message' column as it's no longer needed after computing the sentiment scores
df = df.drop(columns=['Message'])

# Also drop other specified columns
columns_to_drop = ['PostUserId', 'ReplyToForumMessageId', 'Medal', 'MedalAwardDate']
df = df.drop(columns=columns_to_drop)

# Save the modified dataframe back to a CSV file
df.to_csv(output_path, index=False)

print("Cleaning and saving process completed.")


In [8]:
# Cleaning ForumMessages.csv file...
import pandas as pd

# Load your dataset
df = pd.read_csv('/kaggle/working/ForumMessages_Cleaned.csv')
output_path = '/kaggle/working/ForumMessages.csv'

# Drop the 'Message' column as it's no longer needed after computing the sentiment scores
df = df.drop(columns=['Message'])



# Convert 'MessageDate' to date format and remove the time part
df['PostDate'] = pd.to_datetime(df['PostDate']).dt.date

# Drop duplicates across the entire dataframe
df = df.drop_duplicates()

# Define a function to categorize sentiment based on 'polarity_score'
def categorize_sentiment(score):
    if score < 0:
        return 'Negative'
    elif score > 0:
        return 'Positive'
    else:
        return 'Neutral'

# Apply the function to create a new 'sentiment' column
df['sentiment'] = df['polarity_score'].apply(categorize_sentiment)

# Save the modified dataframe back to a CSV file
df.to_csv(output_path, index=False)

print("Cleaning and saving process completed.")


Cleaning and saving process completed.


In [4]:
# Merging Users with MasterProfiles, Master Acheivements data to get locations and Id into one file.
import pandas as pd

# Load the first Excel file
file_path1 = '/kaggle/input/meta-kagglemaster-achievements-snapshot/MasterAchievements.csv'
df1 = pd.read_csv(file_path1)

# Load the second Excel file
file_path2 = '/kaggle/input/meta-kagglemaster-achievements-snapshot/MasterProfiles.csv'
df2 = pd.read_csv(file_path2)

# Load the Users.csv file
users_file_path = '/kaggle/input/meta-kaggle/Users.csv'
users_df = pd.read_csv(users_file_path)

# Merge the two DataFrames on 'UserName'
# This performs a left join by default, getting all records from df1
# and the matching records from df2, based on 'UserName'
# Change 'how' argument to 'inner', 'right', or 'outer' as per your requirement
master_profiles_df = pd.merge(df1, df2, on='UserName', how='left')

# Merge the two DataFrames on 'UserName' to get the 'Id' column from the Users table
# The default merge is an inner join, which will only keep records that have matching 'UserName' in both tables
# If you want to keep all records from MasterProfiles and just add matching Ids from Users, you should use how='left'
merged_df = pd.merge(master_profiles_df, users_df[['Id', 'UserName']], on='UserName', how='left')

# Save the merged DataFrame to a new CSV file
output_path = '/kaggle/working/MasterProfiles.csv'
merged_df.to_csv(output_path, index=False)

print(f"Merged data saved to {output_path}")


Merged data saved to /kaggle/working/MasterProfiles.csv


In [6]:
#Map performance tier to GM, Master, Expert, Contributor, Novice, Staff
import pandas as pd

# Assuming 'Users.csv' is in the Kaggle input directory. Adjust the path if it's different.
users_file_path = '/kaggle/input/users-csv/Users.csv'
output_path = '/kaggle/working/Users.csv'

# Load the Users.csv file
users_df = pd.read_csv(users_file_path)

# Create a mapping of performance tier numbers to names
performance_tier_mapping = {
    0: 'novice',
    1: 'contributor',
    2: 'expert',
    3: 'master',
    4: 'grandmaster',
    5: 'staff'
}

# Map the 'PerformanceTier' to 'TierName'
users_df['TierName'] = users_df['PerformanceTier'].map(performance_tier_mapping)

# Save the updated DataFrame to a new CSV file
users_df.to_csv(output_path, index=False)

print(f"Updated Users data with TierName saved to {output_path}")


Updated Users data with TierName saved to /kaggle/working/Users.csv
